### Working with Financial Data

Publicly traded companies are classified into one of 11 industry sectors, called GICS (Global Industry Classification Standard). Investors use these classes to design and construct portfolios, compare companies, evaluate industries, and isolate regional from global trends.

There are 11 GICS sectors

* Communication services
* Consumer discretionary
* Consumer staples
* Energy
* Financials
* Health care
* Industrials
* Information technology
* Materials
* Real estate
* Utilities


In this notebook we'll work with financial data, grouped by sector, to analyze different trends in companies and sectors listed on the S&P 500. 


Begin by loading the GICS sector mappings. They've been provided to you in a file called `gics.csv`


In [2]:
import pandas as pd
dataGICS = pd.read_csv('gics.csv', delimiter=',', header=0)
dataGICS = dataGICS.rename(columns={'Name':'Sector'})
dataGICS.head()

,Code,Sector
0,50,Communication services
1,25,Consumer discretionary
2,30,Consumer staples
3,10,Energy
4,40,Financials


Load the S&P 500 sector mappings and take a look at the data. The file is called `sp500-sector.csv`

In [3]:
dataSP500 = pd.read_csv('sp500-sector.csv', delimiter=',', header=0)
dataSP500.head()

,Symbol,Name,Sector
0,MMM,3M Company,Industrials
1,AOS,A.O. Smith Corp,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie Inc.,Health Care
4,ACN,Accenture plc,Information Technology


Create a dataframe that has the S&P 500 stock symbols with the appropriate sector number. For example, `3M`, symbol `MMM` is in `industrials` which corresponds to sector 20.

In [4]:
combinedData = pd.merge(dataSP500, dataGICS, on='Sector', how='outer')
combinedData.head()

,Symbol,Name,Sector,Code
0,MMM,3M Company,Industrials,20.0
1,AOS,A.O. Smith Corp,Industrials,20.0
2,AYI,Acuity Brands Inc,Industrials,20.0
3,ALK,Alaska Air Group Inc,Industrials,20.0
4,ALLE,Allegion,Industrials,20.0


Working with strings in pandas can be much slower than working with integers and floats. Securities are typically represented by a security id, a numerical representation of the security. Ids have been provided in the file `sp500-ids.csv`. Load these into a dataframe and then generate a new dataframe that contains the columns `security_id` and `sector`.

In [5]:
dataSP500Ids = pd.read_csv('sp500-ids.csv', delimiter=',', header=0)
dataSP500Ids.head()

,Symbol,Security_id
0,MMM,1
1,AOS,2
2,ABT,3
3,ABBV,4
4,ACN,5


Load the file `sp500-5yr.csv` into a dataframe. This file contains 5 years worth of historical prices for securities listed on the S&P 500. Convert the symbols into security ids, and bring in the sector colums as well.

In [6]:
dataSP5005Yr = pd.read_csv('sp500-5yr.csv', delimiter=',', header=0)
dataSP5005Yr = dataSP5005Yr.rename(columns={'symbol':'Symbol'})
combinedData2 = pd.merge(dataSP5005Yr, combinedData, on='Symbol', how='outer')
combinedData2.head()

,date,open,high,low,close,volume,Symbol,Name,Sector,Code
0,2013-02-08,15.07,15.12,14.63,14.75,8407500.0,AAL,American Airlines Group,Industrials,20.0
1,2013-02-11,14.89,15.01,14.26,14.46,8882000.0,AAL,American Airlines Group,Industrials,20.0
2,2013-02-12,14.45,14.51,14.10,14.27,8126000.0,AAL,American Airlines Group,Industrials,20.0
3,2013-02-13,14.30,14.94,14.25,14.66,10259500.0,AAL,American Airlines Group,Industrials,20.0
4,2013-02-14,14.94,14.96,13.16,13.99,31879900.0,AAL,American Airlines Group,Industrials,20.0


Determine which sectors had the best performance for the past year, past three years and past five years. Note the start and end dates on the dataset.

In [7]:
dataset = combinedData2.copy()
dataset['high low diff'] = dataset['high'] - dataset['low']

pastYearData = dataset[(dataset['date'] >= '2018-01-01') & (dataset['date'] <= '2018-12-31')]
grouped1Year = pastYearData.groupby(['Sector'])['high low diff'].mean()

pastThreeYearData = dataset[(dataset['date'] >= '2016-01-01') & (dataset['date'] <= '2018-12-31')]
grouped3Year = pastThreeYearData.groupby(['Sector'])['high low diff'].mean()

pastFiveYearData = dataset[(dataset['date'] >= '2014-01-01') & (dataset['date'] <= '2018-12-31')]
grouped5Year = pastFiveYearData.groupby(['Sector'])['high low diff'].mean()

groupedCombination = pd.merge(grouped1Year, grouped3Year, on='Sector', how='outer')
groupedCombination1 = pd.merge(groupedCombination, grouped5Year, on='Sector', how='outer')
groupedCombination1.columns = ['Last 1 Year', 'Last 3 Year', 'Last 5 Year']
#print(groupedCombination1)

last1Year = groupedCombination1[(groupedCombination1['Last 1 Year'] == groupedCombination1['Last 1 Year'].max())]
print(last1Year['Last 1 Year'])
print("-------------------")

last3Year = groupedCombination1[(groupedCombination1['Last 3 Year'] == groupedCombination1['Last 3 Year'].max())]
print(last3Year['Last 3 Year'])
print("-------------------")

last5Year = groupedCombination1[(groupedCombination1['Last 5 Year'] == groupedCombination1['Last 5 Year'].max())]
print(last5Year['Last 5 Year'])
print("-------------------")

Sector
Health Care    3.432604
Name: Last 1 Year, dtype: float64
-------------------
Sector
Health Care    2.363567
Name: Last 3 Year, dtype: float64
-------------------
Sector
Health Care    2.334248
Name: Last 5 Year, dtype: float64
-------------------


For each sector, determine the top performer and the bottom performer for the 1 year, 3 year, and 5 year windows.

In [8]:
grouped1YearV2 = pastYearData.groupby(['Sector', 'Symbol'])['high low diff'].mean()
grouped3YearV2 = pastThreeYearData.groupby(['Sector', 'Symbol'])['high low diff'].mean()
grouped5YearV2 = pastFiveYearData.groupby(['Sector', 'Symbol'])['high low diff'].mean()
# print(grouped1YearV2)
# print(grouped3YearV2)
# print(grouped5YearV2)

sectors1Year = grouped1YearV2.index.unique(level='Sector')
for sector in sectors1Year:
    sectorData = grouped1YearV2[sector]
    print(sector)
    print('Top Performer 1 Year')
    print(sectorData[(sectorData == sectorData.max())])
    print('Bottom Performer 1 Year')
    print(sectorData[(sectorData == sectorData.min())])
    print("------")
sectors3Year = grouped1YearV2.index.unique(level='Sector')
for sector in sectors3Year:
    sectorData = grouped3YearV2[sector]
    print(sector)
    print('Top Performer 3 Year')
    print(sectorData[(sectorData == sectorData.max())])
    print('Bottom Performer 3 Year')
    print(sectorData[(sectorData == sectorData.min())])
    print("------")
sectors5Year = grouped5YearV2.index.unique(level='Sector')
for sector in sectors5Year:
    sectorData = grouped5YearV2[sector]
    print(sector)
    print('Top Performer 5 Year')
    print(sectorData[(sectorData == sectorData.max())])
    print('Bottom Performer 5 Year')
    print(sectorData[(sectorData == sectorData.min())])
    print("------")

Consumer Discretionary
Top Performer 1 Year
Symbol
AMZN    37.823462
Name: high low diff, dtype: float64
Bottom Performer 1 Year
Symbol
F    0.259
Name: high low diff, dtype: float64
------
Consumer Staples
Top Performer 1 Year
Symbol
STZ    3.852142
Name: high low diff, dtype: float64
Bottom Performer 1 Year
Symbol
COTY    0.518462
Name: high low diff, dtype: float64
------
Energy
Top Performer 1 Year
Symbol
PXD    4.40555
Name: high low diff, dtype: float64
Bottom Performer 1 Year
Symbol
KMI    0.417323
Name: high low diff, dtype: float64
------
Financials
Top Performer 1 Year
Symbol
BLK    11.425573
Name: high low diff, dtype: float64
Bottom Performer 1 Year
Symbol
HBAN    0.296077
Name: high low diff, dtype: float64
------
Health Care
Top Performer 1 Year
Symbol
MTD    12.756615
Name: high low diff, dtype: float64
Bottom Performer 1 Year
Symbol
BSX    0.631304
Name: high low diff, dtype: float64
------
Industrials
Top Performer 1 Year
Symbol
BA    9.383046
Name: high low diff, dtyp

Imagine two portfolios, one comprised of the securites represented by even ids and one by odd ids. Which portfolio has had the best performance over the 1,3 and 5 year time windows?


In [50]:
from datetime import datetime

allRows = dataset
oddRows = dataset[dataset.index % 2 != 0]
evenRows = dataset[dataset.index % 2 != 1]

def portfolioPerformance(data, portfolio, startDate, endDate, acc):
    dataYearRange = data[(data['date'] >= startDate) & (data['date'] <= endDate)]

    date_format = "%Y-%m-%d"
    a = datetime.strptime(startDate, date_format)
    b = datetime.strptime(endDate, date_format)
    delta = b - a
    mean = dataYearRange['high low diff'].mean()
    acc.append([portfolio, delta.days/365, mean])

portfolioList = []
portfolioPerformance(oddRows, 'odd', '2018-01-01', '2018-12-31', portfolioList)
portfolioPerformance(evenRows, 'even', '2018-01-01', '2018-12-31', portfolioList)
portfolioPerformance(oddRows, 'odd', '2016-01-01', '2018-12-31', portfolioList)
portfolioPerformance(evenRows, 'even', '2016-01-01', '2018-12-31', portfolioList)
portfolioPerformance(oddRows, 'odd', '2014-01-01', '2018-12-31', portfolioList)
portfolioPerformance(evenRows, 'even', '2014-01-01', '2018-12-31', portfolioList)

portfolioColumns = ['portfolio', 'years', 'mean']
pd.DataFrame(portfolioList, columns=portfolioColumns)

,portfolio,years,mean
0,odd,0.99726,2.456652
1,even,0.99726,2.429214
2,odd,3.00000,1.672694
3,even,3.00000,1.669749
4,odd,5.00000,1.601699
5,even,5.00000,1.602867


Combine these two portfolios into a single portfolio. What was the overall performance of the SP500 for the same time periods?


In [52]:
portfolioPerformance(allRows, 'all', '2018-01-01', '2018-12-31', portfolioList)
portfolioPerformance(allRows, 'all', '2016-01-01', '2018-12-31', portfolioList)
portfolioPerformance(allRows, 'all', '2014-01-01', '2018-12-31', portfolioList)

portfolioColumns = ['portfolio', 'years', 'mean']
pd.DataFrame(portfolioList, columns=portfolioColumns)

,portfolio,years,mean
0,all,0.99726,2.442933
1,all,3.00000,1.671222
2,all,5.00000,1.602283
3,all,0.99726,2.442933
4,all,3.00000,1.671222
5,all,5.00000,1.602283
